In [2]:
import os
import requests
import h5py
import numpy as p
from datetime import datetime, timedelta

# Define base URL and file URL for MODIS Snow Cover data
MODIS_SNOW_COVER_URL_TEMPLATE = 'https://nsidc.org/data/mod10a1/1/mod10a1.{}{}.hdf'

# NSIDC credentials (if needed for accessing certain data)
USERNAME = 'skylargale'
PASSWORD = '*Carolyn1928ED'

def download_modis_snow_cover_data(date, download_dir='modis_data'):
    """
    Downloads MODIS snow cover HDF file for a specific date.
    """
    # Get the year and day of year (DOY) from the date
    year = date.year
    day_of_year = date.timetuple().tm_yday  # Day of the year (1-366)

    # Format the date in the required file format (e.g., 2023095 for the 95th day of 2023)
    date_str = f"{year}{day_of_year:03d}"

    # Construct the full URL for the data file
    url = MODIS_SNOW_COVER_URL_TEMPLATE.format(year, date_str)

    # Define the local file path
    local_file_path = os.path.join(download_dir, f"MOD10A1_{date_str}.hdf")

    # Make the request to download the file
    try:
        print(f"Downloading MODIS Snow Cover data for {date_str}...")
        response = requests.get(url, auth=(USERNAME, PASSWORD), stream=True)
        
        if response.status_code == 200:
            os.makedirs(download_dir, exist_ok=True)
            with open(local_file_path, 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            print(f"Downloaded to {local_file_path}")
        else:
            print(f"Failed to download data for {date_str}. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred while downloading data: {e}")

    return local_file_path

def extract_snow_cover_data(hdf_file):
    """
    Extracts snow cover data from the MODIS HDF file.
    """
    try:
        with h5py.File(hdf_file, 'r') as f:
            # List the available groups and datasets in the HDF file
            print(f"Available datasets in {hdf_file}: {list(f.keys())}")
            
            # The snow cover data is typically found in the 'SnowCover' dataset
            # Adjust if needed based on the HDF file structure
            snow_cover_data = f['MODIS_Snow_Cover_Data']['SnowCover'][:]
            print(f"Extracted snow cover data with shape: {snow_cover_data.shape}")
    except Exception as e:
        print(f"An error occurred while reading HDF data: {e}")
        snow_cover_data = None
    
    return snow_cover_data

def compile_snow_cover_data(download_dir='modis_data'):
    """
    Compiles downloaded MODIS snow cover data from multiple HDF files into a single array.
    """
    all_snow_cover = []
    for root, _, files in os.walk(download_dir):
        for file in files:
            if file.endswith(".hdf"):
                file_path = os.path.join(root, file)
                snow_cover_data = extract_snow_cover_data(file_path)
                if snow_cover_data is not None:
                    all_snow_cover.append(snow_cover_data)

    # Combine the list of snow cover data arrays into a single NumPy array
    all_snow_cover_data = np.stack(all_snow_cover, axis=0)
    return all_snow_cover_data

def download_and_compile_data(start_date, end_date):
    """
    Downloads MODIS snow cover data from the start_date to end_date, and compiles it.
    """
    current_date = start_date
    while current_date <= end_date:
        # Step 1: Download MODIS snow cover data for the current date
        download_modis_snow_cover_data(current_date)

        # Move to the next day
        current_date += timedelta(days=1)

    # Step 2: Compile the downloaded data
    all_snow_cover_data = compile_snow_cover_data()
    
    # Step 3: Do something with the compiled data (e.g., save it, process it, etc.)
    print(f"Compiled snow cover data from {start_date} to {end_date} with shape: {all_snow_cover_data.shape}")

def main():
    # Define the start and end date
    start_date = datetime(2000, 2, 23)  # Start from February 23, 2000
    end_date = datetime.today()  # Present day

    # Download and compile MODIS snow cover data from February 23, 2000 to present
    download_and_compile_data(start_date, end_date)

if __name__ == "__main__":
    main()



Failed to download data for 2000054. Status code: 404
Failed to download data for 2000055. Status code: 404
Failed to download data for 2000056. Status code: 404
Failed to download data for 2000057. Status code: 404
Failed to download data for 2000058. Status code: 404
Failed to download data for 2000059. Status code: 404
Failed to download data for 2000060. Status code: 404
Failed to download data for 2000061. Status code: 404
Failed to download data for 2000062. Status code: 404
Failed to download data for 2000063. Status code: 404
Failed to download data for 2000064. Status code: 404
Failed to download data for 2000065. Status code: 404


KeyboardInterrupt: 